In [22]:
import torch
import scipy.io as scio
import tensorflow as tf
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
'''import os   # 指定GPU训练
os.environ["CUDA_VISIBLE_DEVICES"] = "0"    # 使用第一张GPU卡'''

cuda


'import os   # 指定GPU训练\nos.environ["CUDA_VISIBLE_DEVICES"] = "0"    # 使用第一张GPU卡'

In [17]:
path = "../input/my-set/map1.mat"
data = scio.loadmat(path)
map_mat = data['map1']
row, column = map_mat.shape[0], map_mat.shape[1]
map_tensor = torch.FloatTensor(map_mat).unsqueeze(0).unsqueeze(3)
np_map_tensor = map_tensor.numpy()
tf_map_tensor = tf.convert_to_tensor(np_map_tensor)

In [18]:
'构造卷积神经网络模型'
'CNN将形状为(image_height, image_width, color_channels)作为输入,忽略批次大小'
'输入为input_shape,传递给第一层'
model = tf.keras.models.Sequential()
# 第一个参数为卷积核个数,第二个参数为kernel size
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(row, column, 1)))
model.add(tf.keras.layers.AvgPool2D((3, 3)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.AvgPool2D((3, 3)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
# 将卷积网络输出放入Dense层,执行分类
model.add(tf.keras.layers.Flatten())  # Dense层仅接受一维向量,本层进行展平
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(128))  # 输出

print(len(model.layers))

8


In [19]:
print(tf_map_tensor.get_shape())
CNN1 = model(tf_map_tensor)

(1, 3071, 4300, 1)


In [20]:
print(CNN1)

tf.Tensor(
[[-0.01199344  0.03927987 -0.05131172 -0.00738782 -0.09873188  0.0067314
  -0.03554376 -0.00095193  0.02927449  0.0420932  -0.03981532 -0.01967666
  -0.05448858 -0.00655599  0.01888862 -0.00222406 -0.06986507 -0.03380414
   0.00094476 -0.01885448  0.04356729 -0.04197612 -0.10343603 -0.05384873
  -0.05279636  0.0229683  -0.0206384   0.01638739  0.00880728 -0.05545351
  -0.03492679 -0.01913424 -0.04164119 -0.010305    0.01406784  0.01335897
  -0.07806063 -0.00384651 -0.04127924 -0.02634493  0.0057901   0.00409066
  -0.05280922  0.0393883  -0.05598108  0.01877873  0.06095418 -0.04365202
  -0.01247317  0.04166262  0.02410357 -0.01433234 -0.04609297 -0.05157109
   0.01453366 -0.02394102 -0.0212644  -0.00694092  0.00460946  0.04969177
   0.00060853  0.04693786  0.08372839 -0.02615617  0.00876687 -0.03757572
  -0.0613657   0.005762   -0.04183467  0.00949999 -0.02172029 -0.03647587
  -0.03432713 -0.00829653 -0.00633858  0.07522298 -0.01007644 -0.00613527
   0.01459747 -0.03844615  0

In [21]:
!nvidia-smi

Sun May 21 02:58:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    30W /  70W |  14600MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   